## GP Regression on Nodes and Edges in a Graph


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import sys
sys.path.append('..') 
from utils.preprocessing import load_dataset
import torch
import gpytorch
from gpytorch.constraints import Positive

from kernels.kernel_wsn import HodgeDiffusionKernel, HodgeMaternKernel, HodgeDiffusionKernelNonHC, HodgeMaternKernelNonHC
from tqdm.notebook import tqdm 

### Load and preprocess the Dataset

Here we standardize the data around the center and 0 with a standard deviation of 1. This is important when we have node and edge signals which are of different units. 


In [ ]:
data_name = 'water_network'
kernel_name = 'matern'

seed = 2
train_ratio=0.5

incidence_matrices, laplacians, data_train, data_test, data, train_ids, test_ids, hr = load_dataset(data_name, train_ratio=train_ratio, spinors=True, seed=seed)

if data_name in ['water_network']: sc_order = 1

x_train, y_train = torch.from_numpy(data_train[0]).float(), torch.from_numpy(data_train[1]).float()
x_test, y_test = torch.from_numpy(data_test[0]).float(), torch.from_numpy(data_test[1]).float()
x, y = torch.from_numpy(data[0]).float(), torch.from_numpy(data[1]).float()

orig_mean, orig_std = torch.mean(y_train), torch.std(y_train)
y_train = (y_train-orig_mean)/orig_std
y_test = (y_test-orig_mean)/orig_std
y = (y-orig_mean)/orig_std


In [ ]:
if sc_order == 1: 
    incidence_matrices = torch.from_numpy(incidence_matrices).float()
    hr = torch.from_numpy(hr).float()


move data to GPU

In [ ]:
if torch.cuda.is_available():
    print("Using CUDA")
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    if sc_order == 1:
        incidence_matrices = incidence_matrices.cuda()
        hr = hr.cuda()
    x_train, y_train = x_train.cuda(), y_train.cuda()
    x_test, y_test = x_test.cuda(), y_test.cuda()
    x, y = x.cuda(), y.cuda()

# Regression

## kernels
Here we express the kernels in terms of the python variables. 
Given the eigendecompositions:
$$L_0 = U_0 \Lambda_0 U_0^\top \quad L_1 = U_1\Lambda_1 U_1^\top$$

##### diffusion-HC
$$
    K_0= \gamma_{down} U_0 e^{-\kappa_{down} \Lambda_{0}} U_0^\top  \\
    K_1 = \gamma_{up} B_1^\top K_0 B_1 
$$

##### diffusion-NonHC
$$
    K_0 = \gamma_{down} U_0 e^{-\kappa_{down} \Lambda_{0}} U_0^\top \\
    K_1 = \gamma_{up} U_1 e^{-\kappa_{up} \Lambda_{1}} U_1^\top 
$$

##### Matern-HC 
$$
    K_0 = \gamma_{down} U_0 (\frac{2\kappa_{up}}{\kappa_{down}^2} + \Lambda_0) U_0^\top \\
    K_1 = \gamma_{up} B_1^\top K_0 B_1 
$$

##### Matern-NonHC 
$$
    K_0 = \gamma_{down} U_0 (\frac{2\mu_{down}}{\kappa_{down}^2} + \Lambda_0) U_0^\top \\
    K_1 = \gamma_{up} U_1 (\frac{2\mu_{up}}{\kappa_{up}^2} + \Lambda_1) U_1^\top 
$$


### Build a GPR model 

In [ ]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel, mean_function=None):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        if mean_function is None:
            self.mean_module = gpytorch.means.ConstantMean()
        elif mean_function == 'zero':
            self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(kernel, outputscale_constraint=Positive())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x) 
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
if kernel_name == 'diffusion-nonhc':
    kernel = HodgeDiffusionKernelNonHC(incidence_matrices)
elif kernel_name == 'diffusion':
    kernel = HodgeDiffusionKernel(incidence_matrices)
elif kernel_name == 'matern':
    kernel = HodgeMaternKernel(incidence_matrices)
elif kernel_name == 'matern-nonhc':
    kernel = HodgeMaternKernelNonHC(incidence_matrices)
    
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x_train, y_train, likelihood, kernel, mean_function=None)

move model to GPU

In [ ]:
if torch.cuda.is_available():
   model = model.cuda()
   likelihood = likelihood.cuda()

In [ ]:
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:50} value = {param.item()}')

#### Initialize the hyperprameters
we consider random initialization 

In [ ]:
# if kernel_name in ['diffusion','diffusion-nonhc']:
#     hypers = {
#         'likelihood.noise_covar.noise': torch.tensor(1.),
#         'covar_module.base_kernel.kappa_down': torch.tensor(1),
#         'covar_module.base_kernel.kappa_up': torch.tensor(1),
#         'covar_module.base_kernel.gamma_down': torch.tensor(1),
#         'covar_module.base_kernel.gamma_up': torch.tensor(1),
#         'covar_module.outputscale': torch.tensor(1.),
#         'mean_module.raw_constant': orig_mean,
#     }
# elif kernel_name in ['matern','matern-nonhc']:
#     hypers = {
#         'likelihood.noise_covar.noise': torch.tensor(1.),
#         'covar_module.base_kernel.kappa_down': torch.tensor(1),
#         'covar_module.base_kernel.kappa_up': torch.tensor(1),
#         'covar_module.base_kernel.gamma_down': torch.tensor(1),
#         'covar_module.base_kernel.gamma_up': torch.tensor(1),
#         'covar_module.outputscale': torch.tensor(1.),
#         'mean_module.raw_constant': orig_mean,
#     }

# model.initialize(**hypers)


### Train the GPR model 

In [ ]:
training_iter = 1000

In [ ]:
# Wrap training, prediction and plotting from the ExactGP-Tutorial into a function,
# so that we do not have to repeat the code later on
def train(model, likelihood, training_iter=training_iter):
    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    iterator = tqdm(range(training_iter))
    for i in iterator:
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(x_train)
        # Calc loss and backprop gradients
        loss = -mll(output, y_train)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (
                i + 1, training_iter, loss.item()
        ))
        optimizer.step()

#### Train the model the analyze the results

In [ ]:
model.train()
likelihood.train()
train(model, likelihood)

Check model parameters

In [ ]:
model.state_dict()

### Predict


In [ ]:
def predict(model, likelihood, x_test):
    model.eval()
    likelihood.eval()
    # Make predictions by feeding model through likelihood
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        # Test points are regularly spaced along [0,1]
        return likelihood(model(x_test))

In [ ]:
model.eval()
likelihood.eval()
observed_pred = predict(model, likelihood, x_test)

In [ ]:
pred_mean, pred_var = observed_pred.mean, observed_pred.variance
lower, upper = observed_pred.confidence_region()

In [ ]:
test_nodes, test_edges = len(test_ids[0]), len(test_ids[1])

## Metrics

### For nodes 

In [ ]:
mse0 = torch.linalg.norm(y_test[:test_nodes] - pred_mean[:test_nodes])**2/test_nodes
mae0 = gpytorch.metrics.mean_absolute_error(observed_pred[:test_nodes], y_test[:test_nodes])
nlpd0 = gpytorch.metrics.negative_log_predictive_density(observed_pred[:test_nodes], y_test[:test_nodes])

### For edges

In [ ]:
mse1 = torch.linalg.norm(y_test[test_nodes:] - pred_mean[test_nodes:])**2/test_edges
mae1 = gpytorch.metrics.mean_absolute_error(observed_pred[test_nodes:], y_test[test_nodes:] )
nlpd1 = gpytorch.metrics.negative_log_predictive_density(observed_pred[test_nodes:], y_test[test_nodes:])

In [ ]:
pred_all = predict(model, likelihood, x)
all_mean, all_var = pred_all.mean, pred_all.variance

In [ ]:
# print the metrics 
print(f'MSE: {mse0.item():.4f} (node), {mse1.item():.4f} (edge)' '\n'
      f'MAE: {mae0.item():.4f} (node), {mae1.item():.4f} (edge)' '\n'
      f'NLPD: {nlpd0.item():.4f} (node), {nlpd1.item():.4f} (edge)')